<a href="https://colab.research.google.com/github/siriwatsc-debug/DADS5001-Plotly/blob/main/Plotly_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
# DADS5001 : Toos and Programing
# Date : Oct 19, 2026
# Topic : Plotly
# Team : Siriwat, Noppawat and Tanin

import pandas as pd
import plotly.express as px
import seaborn as sns

# Load data
df = pd.read_csv('https://raw.githubusercontent.com/justmarkham/DAT8/master/data/chipotle.tsv', sep ='\t')
df.info()

# convert to (float) and replace $ with ''
df['item_price'] = df['item_price'].str.replace('$', '').astype(float)

df.head(10)
df = df.copy()
df = df.dropna()
df

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4622 entries, 0 to 4621
Data columns (total 5 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   order_id            4622 non-null   int64 
 1   quantity            4622 non-null   int64 
 2   item_name           4622 non-null   object
 3   choice_description  3376 non-null   object
 4   item_price          4622 non-null   object
dtypes: int64(2), object(3)
memory usage: 180.7+ KB


,order_id,quantity,item_name,choice_description,item_price
1,1,1,Izze,[Clementine],3.39
2,1,1,Nantucket Nectar,[Apple],3.39
4,2,2,Chicken Bowl,"[Tomatillo-Red Chili Salsa (Hot), [Black Beans...",16.98
5,3,1,Chicken Bowl,"[Fresh Tomato Salsa (Mild), [Rice, Cheese, Sou...",10.98
7,4,1,Steak Burrito,"[Tomatillo Red Chili Salsa, [Fajita Vegetables...",11.75
...,...,...,...,...,...
4617,1833,1,Steak Burrito,"[Fresh Tomato Salsa, [Rice, Black Beans, Sour ...",11.75
4618,1833,1,Steak Burrito,"[Fresh Tomato Salsa, [Rice, Sour Cream, Cheese...",11.75
4619,1834,1,Chicken Salad Bowl,"[Fresh Tomato Salsa, [Fajita Vegetables, Pinto...",11.25
4620,1834,1,Chicken Salad Bowl,"[Fresh Tomato Salsa, [Fajita Vegetables, Lettu...",8.75


#Question 1 : Find top 5 menu to order

In [7]:
# Top 5 high light menu
total_quantity_per_item = df.groupby('item_name')['quantity'].sum().reset_index()

# Compute top 5 menu to order
top5_items = total_quantity_per_item.nlargest(5, 'quantity')['item_name']
total_quantity_per_item['color'] = total_quantity_per_item['item_name'].apply(
    lambda x: 'highlight_color' if x in top5_items.values else 'default_color'
)

# Set color
color_discrete_map = {'highlight_color': 'blue', 'default_color': 'lightgray'}

# Plot graph
fig = px.bar(total_quantity_per_item.sort_values('quantity', ascending=True),
             x='quantity', y='item_name', orientation='h',
             color='color',
             color_discrete_map=color_discrete_map,
             title='Total Quantity Ordered Per Menu Item (Highlight Top 5)',
             labels={'quantity':'Total Quantity', 'item_name':'Menu Item'})

fig.show()


#Question 2 : Find top 5 menu of average price to order

In [9]:

# Groupby item_name / menu with average
avg_price_per_item = df.groupby('item_name')['item_price'].mean().reset_index()

# Select 5 menu with largest average price
top5_price_items = avg_price_per_item.nlargest(5, 'item_price')['item_name']

# high light top 5 menu
avg_price_per_item['color'] = avg_price_per_item['item_name'].apply(
    lambda x: 'highlight_color' if x in top5_price_items.values else 'default_color'
)

# กำหนดสีสำหรับไฮไลต์และสีพื้นฐาน
color_discrete_map = {'highlight_color': 'mediumseagreen', 'default_color': 'lightgray'}

# สร้างกราฟแท่งพร้อมไฮไลต์สี
fig = px.bar(avg_price_per_item.sort_values('item_price', ascending=True),
             x='item_price', y='item_name', orientation='h',
             color='color', color_discrete_map=color_discrete_map,
             title='Average Price Per Menu Item (Highlight Top 5)',
             labels={'item_price': 'Average Price', 'item_name': 'Menu Item'})

fig.show()



#Question 3 : Find popular menu for customers

In [27]:
# Fill missing values in choice_description with a placeholder
df['choice_description'] = df['choice_description'].fillna('No Description')

# แยก choice_description ออกเป็นหลายตัวเลือก แล้วรวมความถี่
choice_descriptions = df['choice_description'].dropna().str.split(',|;|\[|\]', expand=True).stack().str.strip()
choice_freq = choice_descriptions.value_counts().reset_index()
choice_freq.columns = ['choice', 'count']

# กรอง choice_description ที่ไม่เป็นค่าว่างหรือไม่ใช่ string ว่าง
filtered_choice_descriptions = df[df['choice_description'].str.strip() != '']['choice_description']

# แยก choice_description เป็นหลายตัวเลือก โดยตัดจากตัวคั่น เช่น , ; [ ]
choice_descriptions = filtered_choice_descriptions.str.split(',|;|\[|\]', expand=True).stack().str.strip()

# รวมความถี่ตัวเลือกแต่ละค่า
choice_freq = choice_descriptions.value_counts().reset_index()
choice_freq.columns = ['choice', 'count']

choice_freq.dropna()

# กราฟตัวเลือกยอดนิยม
fig = px.bar(choice_freq[1:].sort_values('count', ascending=True),
             x='count', y='choice', orientation='h',
             title='Top 20 Popular Customer Choices',
             labels={'count':'Count', 'choice':'Choice Description'})
fig.show()

choice_freq.iloc[1:]




<>:5: SyntaxWarning:

invalid escape sequence '\['

<>:14: SyntaxWarning:

invalid escape sequence '\['

<>:5: SyntaxWarning:

invalid escape sequence '\['

<>:14: SyntaxWarning:

invalid escape sequence '\['

/tmp/ipython-input-541063194.py:5: SyntaxWarning:

invalid escape sequence '\['

/tmp/ipython-input-541063194.py:14: SyntaxWarning:

invalid escape sequence '\['



,choice,count
1,Rice,2389
2,Cheese,2281
3,Lettuce,1742
4,Sour Cream,1711
5,Black Beans,1342
6,Fresh Tomato Salsa,1046
7,Guacamole,1037
8,Fajita Vegetables,722
9,Pinto Beans,582
10,Roasted Chili Corn Salsa,457


In [ ]:
# Group by item_name and sum the quantity and price
item_summary = df.groupby('item_name').agg(
    total_quantity=('quantity', 'sum'),
    total_price=('item_price', 'sum')
).reset_index()

# Sort by total quantity to find the most popular items
popular_items = item_summary.sort_values(by='total_quantity', ascending=False)

print("Most Popular Items by Quantity:")
display(popular_items.head(10))

# Sort by total price to find the most expensive items (in terms of total revenue generated)
expensive_items = item_summary.sort_values(by='total_price', ascending=False)

print("\nItems with Highest Total Revenue:")
display(expensive_items.head(10))

# You can also analyze the average price per item if needed
item_summary['average_price'] = item_summary['total_price'] / item_summary['total_quantity']
print("\nItems by Average Price:")
display(item_summary.sort_values(by='average_price', ascending=False).head(10))

Most Popular Items by Quantity:


,item_name,total_quantity,total_price
17,Chicken Bowl,761,7342.73
18,Chicken Burrito,591,5575.82
25,Chips and Guacamole,506,2201.04
39,Steak Burrito,386,3851.43
10,Canned Soft Drink,351,438.75
23,Chips,230,494.34
38,Steak Bowl,221,2260.19
6,Bottled Water,211,302.56
24,Chips and Fresh Tomato Salsa,130,361.36
9,Canned Soda,126,137.34



Items with Highest Total Revenue:


,item_name,total_quantity,total_price
17,Chicken Bowl,761,7342.73
18,Chicken Burrito,591,5575.82
39,Steak Burrito,386,3851.43
38,Steak Bowl,221,2260.19
25,Chips and Guacamole,506,2201.04
21,Chicken Salad Bowl,123,1228.75
22,Chicken Soft Tacos,120,1108.09
45,Veggie Burrito,97,934.77
2,Barbacoa Burrito,91,894.75
44,Veggie Bowl,87,867.99



Items by Average Price:


,item_name,total_quantity,total_price,average_price
42,Steak Salad Bowl,31,343.59,11.083548
15,Carnitas Salad Bowl,6,66.34,11.056667
4,Barbacoa Salad Bowl,10,106.40,10.640000
11,Carnitas Bowl,71,736.71,10.376197
38,Steak Bowl,221,2260.19,10.227104
1,Barbacoa Bowl,66,672.36,10.187273
48,Veggie Salad Bowl,18,182.50,10.138889
5,Barbacoa Soft Tacos,25,250.46,10.018400
3,Barbacoa Crispy Tacos,12,120.21,10.017500
21,Chicken Salad Bowl,123,1228.75,9.989837


In [ ]:
# Clean up the item_price column by removing $ and converting to float
df['item_price'] = df['item_price'].replace('[\$,]', '', regex=True).astype(float)

# Fill missing values in choice_description with a placeholder
df['choice_description'] = df['choice_description'].fillna('No Description')


# Create a treemap: categorize by 'item_name' and 'choice_description'
fig = px.treemap(
    df,
    path=['item_name', 'item_price','choice_description'],       # Hierarchical grouping
    values='item_price',                           # Size by total sales value
    color='item_price',                            # Color scale by sales
    color_continuous_scale='RdBu',
    title='Chipotle Item Hierarchy and Customization'
)
fig.show()

<>:2: SyntaxWarning:

invalid escape sequence '\$'

<>:2: SyntaxWarning:

invalid escape sequence '\$'

/tmp/ipython-input-492109289.py:2: SyntaxWarning:

invalid escape sequence '\$'



In [29]:
# Clean up the item_price column by removing $ and converting to float
df['item_price'] = df['item_price'].replace('[\$,]', '', regex=True).astype(float)

# Fill missing values in choice_description with a placeholder
df['choice_description'] = df['choice_description'].fillna('No Description')


# Create a treemap: categorize by 'item_name' and 'choice_description'
fig = px.treemap(
    df,
    path=['item_name', 'choice_description'],       # Hierarchical grouping
    values='item_price',                           # Size by total sales value
    color='item_price',                            # Color scale by sales
    color_continuous_scale='gnbu',
    title='Chipotle Item Hierarchy and Customization'
)
fig.show()

<>:2: SyntaxWarning:

invalid escape sequence '\$'

<>:2: SyntaxWarning:

invalid escape sequence '\$'

/tmp/ipython-input-375435732.py:2: SyntaxWarning:

invalid escape sequence '\$'



In [ ]:
# Analyze the number of items per order
items_per_order = df.groupby('order_id')['quantity'].sum()

print("\nDistribution of items per order:")
display(items_per_order.describe())

# Visualize the distribution of items per order
fig_items_per_order = px.histogram(items_per_order, x='quantity', title='Distribution of Items per Order')
fig_items_per_order.show()


Distribution of items per order:


,quantity
count,1834.000000
mean,2.711014
std,1.677624
min,1.000000
25%,2.000000
50%,2.000000
75%,3.000000
max,35.000000


In [23]:
# Function to extract choices from the choice_description string
def extract_choices(description):
    if pd.isna(description):
        return []
    # Remove brackets and split by comma
    choices = description.strip("[]").split(", ")
    # Further split choices that contain 'with'
    processed_choices = []
    for choice in choices:
        if ' with ' in choice:
            processed_choices.extend(choice.split(' with '))
        else:
            processed_choices.append(choice)
    return processed_choices

# Apply the function to create a new column with a list of choices
df['individual_choices'] = df['choice_description'].apply(extract_choices)

# Display the first few rows with the new column
display(df[['choice_description', 'individual_choices']].head())

# You can further analyze the individual choices if needed, for example, by exploding the list
 exploded_choices = df.explode('individual_choices')
 display(exploded_choices.head())

df

IndentationError: unexpected indent (ipython-input-4076317791.py, line 23)